<a href="https://colab.research.google.com/github/miguel-peralta/cars_ista322/blob/main/cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cars Relational Databases
ISTA 322 Final Project, Spring 2024 <br>
Miguel Candido Aurora Peralta <br>
## Extract
### KBB Web Scraping
The cars are separated into new and used cars categories. The lists new and used cars are on separate pages. These lists will be combined into one for this project.
#### Used Cars



In [173]:
# Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

First we need to get a list of all of the URLs for the car models on KBB so that more information can be extracted from those pages.

In [174]:
def get_html_doc(url):
  '''Returns the HTML document as a JSON response for the given URL'''
  # requests HTML document for URL
  response = requests.get(url)
  # returns JSON response
  return response.text

In [175]:
def get_kbb_df():
  '''
  Returns a dataframe containing the url, make, model, and year from the
  relative URLs listed on the car models list pages.
  Returns:
    car_info (DataFrame): make, model, year, and url of each model
  '''
  # Create dataframe and lists to store info
  car_info = pd.DataFrame()
  urls = []
  make = []
  model = []
  year = []
  base_url = 'https://www.kbb.com'

  for page in ['new', 'used']:
    url = f'https://www.kbb.com/car-make-model-list/{page}'
    # Create HTML object from url
    html = get_html_doc(url)
    soup = BeautifulSoup(html, 'html.parser')
    # Create list to store relative URLS from the page
    links = []
    # Get all links from the page (the links to models all have the same style)
    for link in soup.find_all('a', attrs={'style':"padding:12px 8px;display:inline-block"}):
      # Add links to the list
      links.append(link.get('href'))
    # Split links using / as delimeter and add information to lists
    for car in links:
      urls.append(base_url+car)
      link_split = car.split('/')
      make.append(link_split[1])
      model.append(link_split[2])
      year.append(link_split[3])

  # Use lists to populate dataframe
  car_info['url'] = urls
  car_info['make'] = make
  car_info['model'] = model
  car_info['year'] = year

  return car_info


In [176]:
kbb = get_kbb_df()

In [209]:
def get_styles(url):
    '''
    Given the URL to a year's model of a car, returns a list of the urls to the
    styles of that model. If there is no style information available, returns a
    1-element list with just the model page URL.
    Args:
      url (string): url to a year's model of a car
    Returns:
      styles (list): list of urls for that model's styles
    '''
    # Create HTML object from url
    html = get_html_doc(url)
    soup = BeautifulSoup(html, 'html.parser')

    # Initialize dataframe
    styles = pd.DataFrame(columns=['style_name', 'combined_fuel_econ', 'seating', 'horsepower', 'cylinders', 'engine_type', 'engine_size_l'])

    # Get style names
    names = soup.find_all('div', attrs={'class' : 'card-heading css-icyrr3'})
    styles['style_name'] = names.get_text()[1:]

    # Extract other style info
    style_info = soup.find_all('div', attrs={'class' : 'css-1g54zqi e151py7u0'})
    style_info_text = [x.get_text() for x in style_info]
    style_info_text = style_info_text[4:]

    mpg_values = style_info_text[::4]
    mpg_values = [x.split()[0] for x in mpg_values]
    styles['combined_fuel_econ'] = mpg_values

    styles['seating'] = style_info_text[1::4]

    styles['horsepower'] = style_info_text[2::4]

    engine = style_info_text[3::4].split(', ')


In [210]:
url = f'https://www.kbb.com/audi/a3/2022/'
print(get_styles_urls(url))

['premium plus sedan 4d', 'premium sedan 4d', 'prestige sedan 4d']


In [211]:
kbb['styles'] = kbb['url'].apply(lambda url: get_styles_urls(url))

In [262]:
url = 'https://www.kbb.com/audi/a3/2015/'
html = get_html_doc(url)
soup = BeautifulSoup(html, 'html.parser')
names = soup.find_all('div', attrs={'class' : 'css-1g54zqi e151py7u0'})
names_text = [x.get_text() for x in names]
print(names_text)
mpg_values = names_text[::4]
seating = names_text[1::4]
print(seating)
horsepower = names_text[2::4]
engine = names_text[3::4]
for i in engine:
  print(i.split(', '))

['Fuel Economy', 'Horsepower', 'Engine', 'Cargo Volume', '27 MPG', '5', '170 @ 4500 RPM', '4-cyl, Turbo, 1.8 Liter', '36 MPG', '5', '150 @ 3500 RPM', '4-cyl, Turbo Diesel, 2.0 Liter', '27 MPG', '5', '170 @ 4500 RPM', '4-cyl, Turbo, 1.8 Liter', '36 MPG', '5', '150 @ 3500 RPM', '4-cyl, Turbo Dsl, 2.0L', '27 MPG', '5', '220 @ 4500 RPM', '4-cyl, Turbo, 2.0 Liter', '27 MPG', '5', '220 @ 4500 RPM', '4-cyl, Turbo, 2.0 Liter', '27 MPG', '5', '220 @ 4500 RPM', '4-cyl, Turbo, 2.0 Liter', '36 MPG', '5', '150 @ 3500 RPM', '4-cyl, Turbo Dsl, 2.0L', '27 MPG', '5', '170 @ 4500 RPM', '4-cyl, Turbo, 1.8 Liter']
['Horsepower', '5', '5', '5', '5', '5', '5', '5', '5', '5']
['Cargo Volume']
['4-cyl', 'Turbo', '1.8 Liter']
['4-cyl', 'Turbo Diesel', '2.0 Liter']
['4-cyl', 'Turbo', '1.8 Liter']
['4-cyl', 'Turbo Dsl', '2.0L']
['4-cyl', 'Turbo', '2.0 Liter']
['4-cyl', 'Turbo', '2.0 Liter']
['4-cyl', 'Turbo', '2.0 Liter']
['4-cyl', 'Turbo Dsl', '2.0L']
['4-cyl', 'Turbo', '1.8 Liter']


## Creating make and model tables

In [188]:
def create_make_table(kbb):
  make = pd.DataFrame(columns = ['make'])
  make_list = sorted(kbb['make'].unique())
  make['make'] = make_list
  make = make.reset_index()
  make = make.rename(columns={'index':'make_id'})
  return make

In [190]:
make = create_make_table(kbb)
print(make.head())

   make_id      make
0        0     acura
1        1      audi
2        2   bentley
3        3       bmw
4        4  cadillac


In [213]:
def create_model_table(kbb, make):
  # model_id, make_id, year, styles
  model = pd.DataFrame(columns=['url', 'make', 'model_name', 'year', 'styles'])
  model['url'] = kbb['url']
  model['make'] = kbb['make']
  model['model_name'] = kbb['model']
  model['year'] = kbb['year']
  model['styles'] = kbb['styles']

  model = pd.merge(model, make, on='make', how='left')
  model = model.drop(columns=['make'])
  model = model.reset_index()
  model = model.rename(columns={'index':'model_id'})
  return model

In [214]:
model = create_model_table(kbb, make)
print(model.head())

   model_id                                url model_name  year  \
0         0  https://www.kbb.com/audi/a3/2025/         a3  2025   
1         1  https://www.kbb.com/audi/a3/2024/         a3  2024   
2         2  https://www.kbb.com/audi/a3/2023/         a3  2023   
3         3  https://www.kbb.com/audi/a4/2024/         a4  2024   
4         4  https://www.kbb.com/audi/a4/2023/         a4  2023   

                                              styles  make_id  
0                                                 []        1  
1                            [premium, premium plus]        1  
2                            [premium, premium plus]        1  
3  [40 tfsi premium, s line 45 tfsi premium, 40 t...        1  
4  [40 tfsi premium, s line 45 tfsi premium, 40 t...        1  


Next we need to retrieve the ends of the urls for the styles for each model of car. Some cars that are too new don't have any styles listed yet. In this case, we will just return an empty array as all of the information that would be contained on the individual style pages is already on the model page.